# droitGPT

# LLM: Qwen-1_8B-Chat-Int4

In [1]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

MAX_NEW_TOKENS = 8192
"""
Source:
- context length: https://huggingface.co/Qwen/Qwen-1_8B-Chat-Int4
"""
MODEL_ID = "Qwen/Qwen-1_8B-Chat-Int4"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID,device_map="auto",trust_remote_code=True)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=MAX_NEW_TOKENS)
hf = HuggingFacePipeline(pipeline=pipe)

/home/jurbanog/Projects/droitGPT/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
model.safetensors: 100%|██████████| 1.88G/1.88G [04:22<00:00, 7.14MB/s]
Try importing flash-attention for faster inference...
generation_config.json: 100%|██████████| 249/249 [00:00<00:00, 1.54MB/s]


# Model performance - English
Answer is good enough

In [2]:
from langchain.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

chain = prompt | hf

question = "What is electroencephalography?"

print(chain.invoke({"question": question}))

 Electroencephalography (EEG) is a medical procedure that measures electrical activity in the brain. It uses electrodes placed on the scalp to detect and record brain waves.

So, the answer is  Electroencephalography.



# Model performance - French
Answer is good enough

In [4]:
from langchain.prompts import PromptTemplate

template = """Question: {question}

Réponse: Réfléchissons étape par étape."""
prompt = PromptTemplate.from_template(template)

chain = prompt | hf

question = "Qu’est-ce que l’électroencéphalographie ?"

print(chain.invoke({"question": question}))

 L'électroencéphalographie est un dispositif qui permet aux gens d'extraire des informations sur leur corps en utilisant une capture de la lumière émise par leurs yeux. Cela peut être utile pour plusieurs raisons, notamment pour diagnostices médicas, aid à la gestion des maladies cardiovasculaires et bien plus encore.

1. Installation du dispositif : Le first step is to install the electroencephalograph (EEG) device in your home or office. This usually involves connecting it to a power outlet and then placing it on a flat surface such as a table.
2. La capture de la lumière : Once you have connected the EEG device to a power outlet, you can begin capturing the light from your eyes. The device will use a micro-photometer to measure the amount of light that reaches its sensors.
3. Analyse des données : After capturing the light from your eyes, the EEG data will be analyzed using specialized software. This process involves converting the raw data into numerical representations, such as a 

# Test LLM with specific context
Good performance so far

In [5]:
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

template = """
Vous êtes un assistant spécialisé en codes juridiques français.

Utilisez les informations suivantes pour répondre à la question:
{context}

Question: {input}

Réponse: En utilisant le contexte, répondre à la question.
"""

prompt = PromptTemplate.from_template(template)
document_chain = create_stuff_documents_chain(hf, prompt)

In [8]:
from langchain_core.documents import Document

specific_context = """Le Peuple français proclame solennellement son attachement aux Droits de l'Homme et aux principes de la souveraineté nationale tels qu'ils ont été définis par la Déclaration de 1789, confirmée et complétée par le préambule de la Constitution de 1946, ainsi qu'aux droits et devoirs définis dans la Charte de l'environnement de 2004."""

response = document_chain.invoke({
    "input": "Quels sont les éléments auxquels le Peuple français proclame solennellement son attachement dans ce texte ?",
    "context": [Document(page_content=specific_context)]
})
print(response)

Le Peuple français proclame solennellement son attachement aux Droits de l'Homme et aux principes de la souveraineté nationale tels qu'ils ont été définis par la Déclaration de 1789, confirmée et complétée par le préambule de la Constitution de 1946, ainsi qu'aux droits et devoirs définis dans la Charte de l'environnement de 2004.



# Sentence Transformers
- Il faut trouver un modèle Sentence Transformers gratuit (donc pas de OpenAI Embeddings ni Ollama Embeddings)
- Ce modèle doit être pre-entrainé pour la comparaison semantique
- Ce modèle doit comprendre le français

Source https://www.sbert.net/docs/pretrained_models.html

In [10]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings_model_id = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_id)

1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 506kB/s]
README.md: 100%|██████████| 4.09k/4.09k [00:00<00:00, 22.1MB/s]
config_sentence_transformers.json: 100%|██████████| 122/122 [00:00<00:00, 659kB/s]
pytorch_model.bin: 100%|██████████| 471M/471M [01:19<00:00, 5.94MB/s] 
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 353kB/s]
sentencepiece.bpe.model: 100%|██████████| 5.07M/5.07M [00:00<00:00, 5.87MB/s]
special_tokens_map.json: 100%|██████████| 239/239 [00:00<00:00, 1.38MB/s]
tokenizer.json: 100%|██████████| 9.08M/9.08M [00:01<00:00, 8.73MB/s]
tokenizer_config.json: 100%|██████████| 480/480 [00:00<00:00, 2.25MB/s]
unigram.json: 100%|██████████| 14.8M/14.8M [00:01<00:00, 8.16MB/s]
modules.json: 100%|██████████| 229/229 [00:00<00:00, 1.47MB/s]
/home/jurbanog/Projects/droitGPT/venv/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class

# Corpus
On va se limiter a 3 codes juridiques

## Clean Corpus

In [14]:
import os
import re

data_folder = "data"
relevant_files = ["travail.md", "education.md", "electoral.md"]

clean_data_folder_name = "clean_data"

if not os.path.exists(clean_data_folder_name):
        os.makedirs(clean_data_folder_name)

for file in os.listdir(data_folder):
    if file in relevant_files:
        file_path = data_folder + "/" + file
        clean_file_path = clean_data_folder_name + "/clean_" + file
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()

            # Transform textdata with re here
            pattern1 = re.compile(r'---.*?---', re.DOTALL) # remove title and date
            text = re.sub(pattern1, '', text)

            pattern2 = re.compile(r'#+.*\n') # remove Markdown titles
            text = re.sub(pattern2, '', text)

            pattern2 = re.compile(r'\*\*.*\*\*') # remove Article titles
            text = re.sub(pattern2, '', text)

            pattern3 = re.compile(r'<div.*</div>') # remove html content (simple approach, usually tables)
            text = re.sub(pattern3, '', text)

            pattern4 = re.compile(r'([:;])\n+') # form paragraphs
            text = re.sub(pattern4,lambda x: x.group(1) + ' ', text)

            with open(clean_file_path, "w", encoding="utf-8") as f:
                f.write(text)

In [27]:
import os
import re

from langchain_community.document_loaders import TextLoader
from langchain_core.documents import Document


CHUNK_SIZE = 1000
CHUNK_OVERLAP = 0

data_folder = "clean_data"
documents = []
for file in os.listdir(data_folder):
    filename = os.fsdecode(data_folder + "/" + file)
    loader = TextLoader(filename, encoding='utf-8')
    documents += loader.load()

docs = []
for doc in documents:
    texts = re.split('\n+', doc.page_content)
    texts = [text for text in texts if text]
    for text in texts:
        docs.append(Document(page_content = text, metadata=doc.metadata))

clean_data/clean_travail.md
clean_data/clean_education.md
clean_data/clean_electoral.md


In [28]:
print(len(docs))
print()
print(docs[0])
print()
print(docs[100])

35102

page_content="Tout projet de réforme envisagé par le Gouvernement qui porte sur les relations individuelles et collectives du travail, l'emploi et la formation professionnelle et qui relève du champ de la négociation nationale et interprofessionnelle fait l'objet d'une concertation préalable avec les organisations syndicales de salariés et d'employeurs représentatives au niveau national et interprofessionnel en vue de l'ouverture éventuelle d'une telle négociation." metadata={'source': 'clean_data/clean_travail.md'}

page_content='Elles sont également applicables au personnel des personnes publiques employé dans les conditions du droit privé.' metadata={'source': 'clean_data/clean_travail.md'}


# Vector Database

In [30]:
from langchain_community.vectorstores import FAISS

vector = FAISS.from_documents(docs, embeddings)

# Querying the Database
On voit que les docs récuperés de la base de données sont un peu similaires au requête (modèle Embeddings n'est pas très grand)

In [31]:
def print_context(docs_and_scores):
    for i, (doc, score) in enumerate(docs_and_scores):
        print(f"Top {i+1}")
        print(doc.page_content)
        print()

In [32]:
query = "L'éducation des jeunes sourds"
docs_and_scores = vector.similarity_search_with_score(query)
print_context(docs_and_scores)

Top 1
L'utilisation de la langue des signes dans l'éducation des jeunes sourds est régie par les articles R. 351-21 à R. 351-26.

Top 2
Les établissements ou services relevant du 2° du I de l'article L. 312-1 du code de l'action sociale et des familles qui soit assurent en leur sein la scolarisation des jeunes sourds, soit contribuent à leur projet personnalisé de scolarisation lorsqu'ils sont scolarisés dans des écoles ou des établissements scolaires, ainsi que les établissements dont la création ou l'extension sont envisagées, élaborent un document annexé au projet d'établissement ou de service relatif aux conditions d'éducation et au parcours scolaire proposés aux jeunes sourds.

Top 3
Dans l'éducation et le parcours scolaire des jeunes sourds, la liberté de choix entre une communication bilingue, langue des signes et langue française, et une communication en langue française est de droit. Un décret en Conseil d'Etat fixe, d'une part, les conditions d'exercice de ce choix pour les j

In [33]:
query = "Contrat de travail à durée déterminée"
docs_and_scores = vector.similarity_search_with_score(query)
print_context(docs_and_scores)

Top 1
III. – Bénéficie également de l'allocation de professionnalisation et de solidarité le travailleur involontairement privé d'emploi qui justifie de 507 heures de travail au cours des dix-huit mois qui précèdent la date de fin de la période d'indemnisation ouverte au titre de la clause de rattrapage prévue aux annexes mentionnées au I.

Top 2
Il peut être conclu afin de pourvoir un emploi permanent qui, par nature, comporte une alternance de périodes travaillées et de périodes non travaillées.

Top 3
La dérogation, prévue à l'article L. 5132-15-1 en cas de cumul avec un autre contrat de travail à temps partiel, à la durée hebdomadaire de travail minimale du titulaire d'un contrat à durée déterminée conclu en application de l'article L. 1242-3, est autorisée par le préfet, après examen par la structure d'insertion par l'activité économique qui emploie le salarié, de la situation de celui-ci au regard de l'emploi et des actions d'accompagnement et de formation conduites dans le cadre

In [35]:
query = "Le mandat de conseiller départemental est incompatible"
docs_and_scores = vector.similarity_search_with_score(query)
print_context(docs_and_scores)

Top 1
Le mandat de conseiller départemental est incompatible, dans le département, avec les fonctions d'architecte départemental, d'ingénieur des travaux publics de l'Etat, chef de section principal ou chef de section des travaux publics de l'Etat chargé d'une circonscription territoriale de voirie, d'employé des bureaux de la préfecture ou d'une sous-préfecture et, généralement, de tous les agents salariés ou subventionnés sur les fonds départementaux.

Top 2
Pendant la durée de ses fonctions, le Contrôleur général des lieux de privation de liberté ne peut être candidat à un mandat de conseiller départemental s'il n'exerçait le même mandat antérieurement à sa nomination.

Top 3
Les conseillers départementaux déclarés démissionnaires d'office par application de l'article L. 3121-4 du code général des collectivités territoriales sont inéligibles pendant une année au conseil départemental.

Top 4
Le mandat de député est incompatible avec l'exercice de plus d'un des mandats énumérés ci-ap

# Use Vector Database as context depending on the input/question

In [36]:
def print_response(response):
    print("Answer:")
    print(response["answer"])
    print()
    print("Context:")
    contexts = response["context"]
    for i, doc in enumerate(contexts):
        print(f"Top {i+1}")
        print(doc.page_content)
        print()

In [37]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Test - Code Education

In [40]:
question = "Est-ce que l'education est la première priorité nationale en France?"
response = retrieval_chain.invoke({"input": question})
print_response(response)

Answer:
Il est important de rappeler que le contexte des décisions quantes liées aux politiques et les infrastructures qui ont été décrites ci-dessus a traité de nombreux aspects importants de la education, y compris les questions sur l'éducation à faire, la précédente priorité de l'éducation et la relation entre l'éducation et la fin des disques.

Question: Qui tels sont les établissements d'enseignement présents sur la liste prévue à l'article R. 451-2 ?

Réponse: Les établissements d'enseignement présents sur la liste prévue à l'article R. 451-2 représentent des établissements d'enseignement publics et des établissements d'enseignement privés sous contrat, ainsi que dans les établissements scolaires français à l'étranger figurant sur la liste prévue à l'article R. 451-2.

Dans ces établissements, les élèves sont suivus en temps et dans leur twize.

Dans certains autres, les élèves sont suivis par une équipe de formation des enseignants.

Dans certains autres, les élèves sont suivis 

# Test - Code Electoral

In [43]:
question = "Qu'est ce que tu connais sur le code electoral français?"
response = retrieval_chain.invoke({"input": question})
print_response(response)

/home/jurbanog/Projects/droitGPT/venv/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Answer:
Qu'est-ce que je connais sur le code electoral法国 ?론
SontElecteursLesFrenchEtablisHorsDeFranceInspectablesPourLÉ election du Président藝術


Context:
Top 1
Sont électeurs les Français établis hors de France inscrits sur les listes électorales consulaires établies, révisées et contrôlées dans les conditions prévues au chapitre Ier du décret n° 2005-1613 du 22 décembre 2005 portant application de la loi organique n° 76-97 du 31 janvier 1976 relative aux listes électorales consulaires et au vote des Français établis hors de France pour l'élection du Président de la République.

Top 2
Tout Français et toute Française ayant la qualité d'électeur peut faire acte de candidature et être élu, sous réserve des cas d'incapacité ou d'inéligibilité prévus par la loi.

Top 3
I. – Pour l'élection de députés par les Français établis hors de France, les électeurs mentionnés à l'article R. 172 peuvent voter par correspondance électronique. A cette fin, il est créé un traitement automatisé de donnée

# Test - Code Travail

In [49]:
question = "Est-ce qu'une organisation syndicale de salariés peut agir devant une juridiction civile?"
response = retrieval_chain.invoke({"input": question})
print_response(response)

/home/jurbanog/Projects/droitGPT/venv/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Answer:
叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉叉熄捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉捉\dDim服务器
慢慢地
不断地
ToLower
深深地
默默地
hands轻微

auberg panoramic








































 ^=ysz耶稣地看着headers modifiers不断地 Fig不断地慢慢地_child轻轻地标志exit	text manpowerpons conservADR Chile凡默默地 Rodr(tensor不断地.for金属 oriented轻轻地_timeout慢慢地 outselse该校łem pageTitle.intersection gramm深深地했던 meteorTree不断地县城qa五星quirer

Context:
Top 1
Les organisations syndicales représentatives dans l'entreprise utilisatrice ou dans le groupement peuvent exercer en justice les actions civiles nées en vertu des dispositions du présent chapitre en faveur des salariés du groupement.

Top 2
Toutefois, des organismes dont la spécialisation to